In [1]:
import os
import numpy as np
from PIL import Image
import glob
import pandas as pd

import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torch.autograd import Variable

class DatasetProcessing(Dataset):
    def __init__(self, file_list, transform=None):
        self.img_filename = file_list
        self.transform = transform        

    def __getitem__(self, index):
        img = Image.open(self.img_filename[index])
        filename = self.img_filename[index]
        img = img.convert('RGB')
        if self.transform is not None:
            img = self.transform(img)        
        return img, filename
    def __len__(self):
        return len(self.img_filename)


/home/pramod/.local/lib/python3.5/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# for testimgs
DATA_PATH = './imgs/'
SAVE_PATH = './test_features'

use_gpu = True

file_list = glob.glob(DATA_PATH+'*.jpg')

print(len(file_list))
file_list[0]

import torchvision.models as models
model = models.resnet101(pretrained=True)

"""
# remove last fully-connected layer -- for alexnet
new_classifier = nn.Sequential(*list(model.classifier.children())[:-1])
model.classifier = new_classifier
"""

# remove last fully-connected layer - for resnet like models 
modules=list(model.children())[:-1]
model=nn.Sequential(*modules)

#print(model)

# for evaluation 
model.eval()

if use_gpu:
    print('yes')
    model = model.cuda()

batch_size = 32
workers = 4


22
yes


In [3]:
#file_list[0]

a = file_list[0]
print(a)
print(a[len(DATA_PATH):])

./imgs/img3.jpg
img3.jpg


In [4]:
scaler = transforms.Resize((224, 224))
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
to_tensor = transforms.ToTensor()

transformations = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    normalize])

In [5]:
dset_test = DatasetProcessing(file_list, transformations)

test_loader = DataLoader(dset_test,
                         batch_size=batch_size,
                         shuffle=False,
                         num_workers=4
                        )

dset_test.__len__()

22

In [22]:
epochs = 1
feature_df = pd.DataFrame()
feature_dict = {}

for epoch in range(epochs):
    ### feature extraction phase        
    for iter, testdata in enumerate(test_loader, 0):
        print("*********iter:" + str(iter) + ' *********')
        test_inputs, test_file_name = testdata
        
        if use_gpu:
            test_inputs = Variable(test_inputs.cuda())
        else: test_inputs = Variable(test_inputs)
            
        test_outputs = model(test_inputs)
        test_outputs = test_outputs.cpu().data.numpy()        
        
        # this is not required for the alexnet 
        # for resnet to squeeze the dimension from the avg pool layers
        test_outputs = np.squeeze(test_outputs)
        # sometimes due to the selection of a particular batch size 
        # the output becomes a 1d vector instead of bsize X dim 2d vector
        if (len(test_outputs.shape) != 2):
            test_outputs = np.reshape(test_outputs,(1, -1))
        
        for i in range(test_outputs.shape[0]):

            # prepare the saving path
            temp = test_file_name[i]
            temp = temp[len(DATA_PATH):]
            index = temp.find(".jpg")
            temp = temp[:index]
            feature_dict[str(temp)] = test_outputs[i,:]
            feature_df = pd.DataFrame(feature_dict)
            
            
            #save_file_name = SAVE_PATH + temp
            #print(save_file_name)
            
            #np.savetxt(save_file_name, test_outputs[i,:])
feature_df = feature_df.transpose()

*********iter:0 *********


In [50]:
csv_file = "./test_imgs_features.csv"
feature_df.to_csv(csv_file)